In [1]:
import pandas as pd
import regex as re


In [2]:
# Converting CSV files into Pandas dataframes
df_player = pd.read_csv('Resources/players.csv')
df_player_att = pd.read_csv('Resources/players_attributes.csv')
df_team = pd.read_csv('Resources/team.csv')
df_team_att = pd.read_csv('Resources/team_attributes.csv')
df_fifa_players = pd.read_csv('Resources/fifa_players.csv')
df_player_gamestats = pd.read_csv('Resources/final_data.csv')



In [3]:
#Getting distinct values based on latest date of recorded statistics of each Team.
df_team_att['date'] = pd.to_datetime(df_team_att['date'])
df_team_att['date'].dt.date
df_team_att

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,1454,15005,10000,2011-02-22,52,Balanced,NaN,Little,52,Mixed,...,53,Normal,Organised,46,Medium,48,Press,53,Normal,Cover
1454,1455,15005,10000,2012-02-22,54,Balanced,NaN,Little,51,Mixed,...,50,Normal,Organised,44,Medium,55,Press,53,Normal,Cover
1455,1456,15005,10000,2013-09-20,54,Balanced,NaN,Little,51,Mixed,...,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover
1456,1457,15005,10000,2014-09-19,54,Balanced,42.0,Normal,51,Mixed,...,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover


In [4]:
# Merging Team information data with Team Attibutes
df_team_combined = pd.merge(df_team, df_team_att, how='inner', on='team_api_id')

# Filtering Data to show only results of the most recent date
latest_indices = df_team_combined.groupby('team_long_name')['date'].idxmax()
latest_team_df = df_team_combined.loc[latest_indices]

# Replacing Null Values using Fillna , Replacing Buildup play dribbling with 0 where class is "Little"
latest_team_df['buildUpPlayDribbling'].fillna(0, inplace=True)

C:\Users\rdesh\AppData\Local\Temp\ipykernel_16392\2867011828.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  latest_team_df['buildUpPlayDribbling'].fillna(0, inplace=True)


In [5]:
# Defining function to extract the last name from dataframes to have a common element for merging. 
def extract_last_name(name):
    # Use regex to find the last word in the string, which is assumed to be the last name
    last_name = re.search(r'(\w+)$', name)
    
    # Return the last name if a match is found
    if last_name:
        return last_name.group(1)
    else:
        return None

In [6]:
# This Df is a complete set with progressional growth over the years of each player. 
df_player_comp = pd.merge(df_player,df_player_att, how='left', on='player_api_id' ) 

# Getting distinct values based on latest date of recorded statistics of each player. 
df_player_comp['date'] = pd.to_datetime(df_player_comp['date'])
latest_indices = df_player_comp.groupby('player_name')['date'].idxmax()
latest_value = df_player_comp.loc[latest_indices]

latest_value['birthday'] = pd.to_datetime(latest_value['birthday'])
latest_value['birthday'] = latest_value['birthday'].dt.date

In [7]:
# Using function to extract last name using Regex. This way we have a common element in both datasets and can use that to merge. 
latest_value['last_name'] = latest_value['player_name'].apply(extract_last_name)
latest_value


,id_x,player_api_id,player_name,player_fifa_api_id_x,birthday,height,weight,id_y,player_fifa_api_id_y,date,...,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,last_name
0,1,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,1,218353,2016-02-18,...,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,Appindangoye
5,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146,6,189615,2016-04-21,...,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0,Cresswell
38,3,162549,Aaron Doran,186170,1991-05-13,170.18,163,39,186170,2016-01-07,...,61.0,23.0,22.0,24.0,16.0,11.0,12.0,9.0,13.0,Doran
64,4,30572,Aaron Galindo,140161,1982-05-08,182.88,198,65,140161,2016-04-21,...,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0,Galindo
87,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,45.0,75.0,73.0,71.0,8.0,6.0,16.0,12.0,11.0,Hughes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183953,11073,36491,Zsolt Low,111191,1979-04-29,180.34,154,183954,111191,2011-02-22,...,61.0,67.0,71.0,59.0,15.0,13.0,5.0,6.0,6.0,Low
183960,11074,35506,Zurab Khizanishvili,47058,1981-10-06,185.42,172,183961,47058,2011-02-22,...,45.0,65.0,66.0,65.0,14.0,7.0,15.0,10.0,12.0,Khizanishvili
183968,11075,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,183969,102359,2012-08-31,...,78.0,22.0,31.0,30.0,9.0,13.0,8.0,15.0,5.0,Misimovic
41617,2483,11833,de Oliveira Cleber Monteiro,163001,1980-05-23,182.88,170,41618,163001,2012-02-22,...,51.0,53.0,64.0,61.0,15.0,15.0,15.0,12.0,14.0,Monteiro


In [8]:
# Using function to extract last name using Regex. This way we have a common element in both datasets and can use that to merge. 
df_fifa_players.rename(columns= {'birth_date' : 'birthday'}, inplace=True)
df_fifa_players['last_name'] = df_fifa_players['name'].apply(extract_last_name)

In [9]:
# combining dataframes to cummalate all relevant columns 
df_combined = pd.merge(latest_value, df_fifa_players, how= 'inner', on='last_name')
df_combined

,id_x,player_api_id,player_name,player_fifa_api_id_x,birthday_x,height,weight,id_y,player_fifa_api_id_y,date,...,long_shots_y,aggression_y,interceptions_y,positioning_y,vision_y,penalties_y,composure,marking_y,standing_tackle_y,sliding_tackle_y
0,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146,6,189615,2016-04-21,...,62,73,75,60,68,59,71,72,78,79
1,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,30,70,65,41,35,38,49,68,70,64
2,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,27,69,62,45,38,45,56,64,63,61
3,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,26,66,63,29,41,45,70,65,65,63
4,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154,88,17725,2015-12-24,...,37,63,64,23,46,41,63,66,65,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18822,2483,11833,de Oliveira Cleber Monteiro,163001,1980-05-23,182.88,170,41618,163001,2012-02-22,...,69,73,73,60,67,65,72,56,68,62
18823,2775,38804,dos Santos Fabio Junior,166048,1982-10-06,175.26,174,46290,166048,2011-02-22,...,46,43,20,62,61,46,73,19,25,22
18824,2775,38804,dos Santos Fabio Junior,166048,1982-10-06,175.26,174,46290,166048,2011-02-22,...,61,31,31,62,64,64,66,17,32,31
18825,2775,38804,dos Santos Fabio Junior,166048,1982-10-06,175.26,174,46290,166048,2011-02-22,...,35,63,51,27,32,48,56,78,74,52


In [10]:
# Dropped duplicate values from dataframe
df_combined.drop_duplicates(subset='player_name', inplace=True)


In [11]:
# Dropping Duplicate and irrelevant columns
df_combined.drop(columns=['player_fifa_api_id_x','weight', 'id_y', 'player_fifa_api_id_y','date','attacking_work_rate', 'defensive_work_rate','last_name', 'name',
       'full_name', 'birthday_y', 'age', 'height_cm','overall_rating_y', 'potential_y','preferred_foot_y',
       'international_reputation(1-5)', 'weak_foot(1-5)', 'skill_moves(1-5)',
       'body_type', 'release_clause_euro', 'national_team', 'national_rating',
       'national_team_position', 'national_jersey_number', 'crossing_y',
       'finishing_y', 'heading_accuracy_y', 'short_passing_y', 'volleys_y',
       'dribbling_y', 'curve_y', 'freekick_accuracy', 'long_passing_y',
       'ball_control_y', 'acceleration_y', 'sprint_speed_y', 'agility_y',
       'reactions_y', 'balance_y', 'shot_power_y', 'jumping_y', 'stamina_y',
       'strength_y', 'long_shots_y', 'aggression_y', 'interceptions_y',
       'positioning_y', 'vision_y', 'penalties_y', 'composure', 'marking_y',
       'standing_tackle_y', 'sliding_tackle_y'], inplace=True)

In [20]:
# Removed Suffixes ('_x' and '_y') created by the merges 
df_combined.rename(columns=lambda x: x.replace('_x', '').replace('_y', ''), inplace=True)
df_combined.columns

Index(['id', 'player_api_id', 'player_name', 'birthday', 'height',
       'overall_rating', 'potential', 'preferred_foot', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving',
       'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes',
       'weight_kgs', 'positions', 'nationality', 'value_euro', 'wage_euro'],
      dtype='object')

In [27]:
# Removed Null values within the dataframe.
df_cleaned = df_combined.dropna(subset=['wage_euro','value_euro','volleys','curve','agility','balance','jumping'])


In [29]:
# final check for null values
df_cleaned.isnull().sum()

id                    0
player_api_id         0
player_name           0
birthday              0
height                0
overall_rating        0
potential             0
preferred_foot        0
crossing              0
finishing             0
heading_accuracy      0
short_passing         0
volleys               0
dribbling             0
curve                 0
free_kick_accuracy    0
long_passing          0
ball_control          0
acceleration          0
sprint_speed          0
agility               0
reactions             0
balance               0
shot_power            0
jumping               0
stamina               0
strength              0
long_shots            0
aggression            0
interceptions         0
positioning           0
vision                0
penalties             0
marking               0
standing_tackle       0
sliding_tackle        0
gk_diving             0
gk_handling           0
gk_kicking            0
gk_positioning        0
gk_reflexes           0
weight_kgs      

In [30]:
# Converting Birthday Colum to datetime format. 
df_cleaned['birthday'] = df_cleaned['birthday'].astype('datetime64[ns]')

C:\Users\rdesh\AppData\Local\Temp\ipykernel_16392\2959105930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['birthday'] = df_cleaned['birthday'].astype('datetime64[ns]')


In [31]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5510 entries, 0 to 18823
Data columns (total 46 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  5510 non-null   int64         
 1   player_api_id       5510 non-null   int64         
 2   player_name         5510 non-null   object        
 3   birthday            5510 non-null   datetime64[ns]
 4   height              5510 non-null   float64       
 5   overall_rating      5510 non-null   float64       
 6   potential           5510 non-null   float64       
 7   preferred_foot      5510 non-null   object        
 8   crossing            5510 non-null   float64       
 9   finishing           5510 non-null   float64       
 10  heading_accuracy    5510 non-null   float64       
 11  short_passing       5510 non-null   float64       
 12  volleys             5510 non-null   float64       
 13  dribbling           5510 non-null   float64       
 

In [19]:
#exporting Dataframes to Csv 
df_player.to_csv('Resources/players.csv', index=False)
df_player_att.to_csv('Resources/players_attributes.csv', index=False)
df_team.to_csv('Resources/team.csv', index=False)
df_team_att.to_csv('Resources/team_attributes.csv', index=False)
df_cleaned.to_csv('Resources/cleaned_data.csv', index=False)